In [3]:
#import and clean data
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go 
import plotly.subplots as make_subplots

df = pd.read_csv(
    '/Users/levankikadze/Desktop/Bolt_Food/Pandas/BoltFood/churningUsers/Data/UserOrdersTotal_past1yr_Breakdown_ShulaGiven.csv',
    dtype={'Spend Objective': str, 'Discount Value Local': 'float'},
    thousands=',',
    )

df.dropna(subset=['Order ID'], inplace=True)
df[['Order ID', 'User ID', 'Provider ID', 'Vendor ID']] = df[['Order ID', 'User ID', 'Provider ID', 'Vendor ID']].astype(int)
df['Order Created Date'] = pd.to_datetime(df['Order Created Date'])

#the data was fetched since 2024-01-01, so we filter out orders before July 28, 2024
cutoff = pd.Timestamp('2024-07-28')
df = df.loc[df['Order Created Date'] >= cutoff].copy()



In [4]:
campaigns = df['Spend Objective'].astype(str).str.lower()

conditions = [
    campaigns.isin(['activation', 'reactivation', 'churn', 'engagement', 'acquisition', 'experiment']),
    campaigns == 'provider_campaign_portal',
    campaigns.str.startswith('provider_campaign_'),
    campaigns == 'marketing',
    campaigns.str.startswith('sp_'),
    campaigns == 'bolt_plus_campaign',
    campaigns == 'other',
    campaigns == 'NaN'
]

choices = [
    'marketplace_campaigns',
    'portal_campaigns',
    'am_campaigns',
    'marketing',
    'smart_promotions',
    'bolt_plus',
    'other_campaigns',
    'no_campaign'
]

df['Spend Objective'] = np.select(conditions, choices, default='no_campaign')


In [5]:
#Deduplicate the DataFrame
#Assign priority to each campaign type
priority = {
    'marketing': 1,
    'am_campaigns': 2,
    'portal_campaigns': 3, 
    'smart_promotions': 4,
    'marketplace_campaigns': 5,
    'bolt_plus': 6,
    'other_campaigns': 7,
    'no_campaign': 8,
}

# Map the priority to the DataFrame
df['Priority'] = df['Spend Objective'].map(priority)

# Sort the DataFrame by 'User ID' and 'Priority'
df.sort_values(by=['Order ID', 'Priority'], inplace=True)

# Drop duplicates, keeping the first occurrence
df_deduplicated = (
    df.drop_duplicates(subset=['Order ID'], keep='first')
      .copy()
)


In [6]:
#Run the last order analysis on the deduplicated DataFrame
df_last_orders = (df_deduplicated.sort_values(by=['User ID', 'Order Created Date'], ascending=False).drop_duplicates(subset=['User ID'], keep='first'))
# Make the needed columns
df_last_orders['days_since_last_order'] = (
    pd.Timestamp(datetime.today().date()) -
    df_last_orders['Order Created Date']
).dt.days
df_last_orders['used_discount'] = df_last_orders['Discount Value Local'] > 0

# Define bins & labels
bins   = [0, 30, 60, 90, 120, np.inf]
labels = ['0-30', '31-60', '61-90', '91-120', '120+']

df_last_orders['recency_group'] = pd.cut(
    df_last_orders['days_since_last_order'],
    bins=bins,
    labels=labels,
    right=True
)

# Group & pivot for counts
counts = (
    df_last_orders
      .groupby(['recency_group', 'used_discount'], observed=True)
      .size()
      .unstack(fill_value=0)
      .rename(columns={False:'No Discount', True:'Used Discount'})
)

# (4) Calculate percentages
pct = counts.div(counts.sum(axis=1), axis=0).mul(100).round(1)
pct.columns = [c + ' %' for c in pct.columns]

# (5) Combine into one clean table
summary = counts.join(pct)
print(summary)


# (6) Breakdown of discount types among those who used a discount
discount_breakdown = (
    df_last_orders[df_last_orders['used_discount']]       # filter to only “used discount”
      .groupby(['recency_group', 'Spend Objective'])
      .size()
      .unstack(fill_value=0)
)

# (7) Turn those into percentages of the “used discount” total per bucket
discount_pct = (
    discount_breakdown
      .div(discount_breakdown.sum(axis=1), axis=0)
      .mul(100)
      .round(1)
      .add_suffix(' %')
)

# (8) Put counts and % side by side
discount_summary = discount_breakdown.join(discount_pct)

print(discount_summary)


               No Discount  Used Discount  No Discount %  Used Discount %
recency_group                                                            
0-30                 11692          32150           26.7             73.3
31-60                 7246          20934           25.7             74.3
61-90                 4723          15341           23.5             76.5
91-120                4339          12620           25.6             74.4
120+                 24923          68034           26.8             73.2
Spend Objective  am_campaigns  bolt_plus  marketing  marketplace_campaigns  \
recency_group                                                                
0-30                    14728       2028        141                  10317   
31-60                    5758        831        362                  10950   
61-90                    5831        116        796                   6435   
91-120                   3912          0        680                   6190   
120+          

/var/folders/q4/9fmh2gnj57lgm7stdq1z_66c0000gn/T/ipykernel_43522/3002465613.py:42: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(['recency_group', 'Spend Objective'])


In [7]:
# run the same analysis on one time users
# — Identify users with exactly one order —
order_counts = df_deduplicated['User ID'].value_counts()
single_users = order_counts[order_counts == 1].index

# — Filter to only those single‑order rows —
df_single = df_deduplicated[df_deduplicated['User ID'].isin(single_users)].copy()

# — Now rerun your “last order” steps on df_single —
# (since each user has only one order, it already is their “last”)
df_single['days_since_last_order'] = (
    pd.Timestamp(datetime.today().date()) -
    df_single['Order Created Date']
).dt.days
df_single['used_discount'] = df_single['Discount Value Local'] > 0

# — Bin into recency groups —
df_single['recency_group'] = pd.cut(
    df_single['days_since_last_order'],
    bins=[0,30,60,90,120,np.inf],
    labels=['0-30','31-60','61-90','91-120','120+'],
    right=True
)

# — 1) Counts & percentages of discount usage —
counts_single = (
    df_single
      .groupby(['recency_group','used_discount'], observed=True)
      .size()
      .unstack(fill_value=0)
      .rename(columns={False:'No Discount', True:'Used Discount'})
)
pct_single = counts_single.div(counts_single.sum(axis=1), axis=0).mul(100).round(1)
pct_single.columns = [c + ' %' for c in pct_single.columns]
summary_single = counts_single.join(pct_single)
print(summary_single)

# — 2) Breakdown of which campaign‑type discounts were used —
discount_breakdown_single = (
    df_single[df_single['used_discount']]
      .groupby(['recency_group','Spend Objective'], observed=True)
      .size()
      .unstack(fill_value=0)
)
discount_pct_single = (
    discount_breakdown_single
      .div(discount_breakdown_single.sum(axis=1), axis=0)
      .mul(100)
      .round(1)
      .add_suffix(' %')
)
discount_summary_single = discount_breakdown_single.join(discount_pct_single)
print(discount_summary_single)



               No Discount  Used Discount  No Discount %  Used Discount %
recency_group                                                            
0-30                  1326           5126           20.6             79.4
31-60                 1257           4900           20.4             79.6
61-90                  856           4982           14.7             85.3
91-120                 757           5185           12.7             87.3
120+                  7840          37560           17.3             82.7
Spend Objective  am_campaigns  bolt_plus  marketing  marketplace_campaigns  \
recency_group                                                                
0-30                     1967         43         16                   2549   
31-60                    1025         42        308                   2957   
61-90                    1483         21        712                   2237   
91-120                   1229          0        640                   2764   
120+          

In [8]:
# 0) Ensure clean categories (optional but neat)
spend_order = [
    'marketplace_campaigns','portal_campaigns','am_campaigns','marketing',
    'smart_promotions','bolt_plus','other_campaigns','no_campaign'
]
df_last_orders['Spend Objective'] = (
    df_last_orders['Spend Objective']
    .fillna('no_campaign')
    .astype(pd.CategoricalDtype(spend_order, ordered=True))
)

# 1) Counts table (no filter!)
counts_so = (
    df_last_orders
      .groupby(['recency_group','Spend Objective','used_discount'], observed=True)
      .size()
      .unstack('used_discount', fill_value=0)
      .rename(columns={False:'No Discount', True:'Used Discount'})
)

# 2) % table (row-wise within each recency_group × spend objective)
pct_so = (
    counts_so
      .div(counts_so.sum(axis=1), axis=0)
      .mul(100).round(1)
      .add_suffix(' %')
)

# 3) Final summary
summary_so = counts_so.join(pct_so)
print(summary_so)          # MultiIndex rows
summary_so_reset = summary_so.reset_index()
print(summary_so_reset)


used_discount                        No Discount  Used Discount  \
recency_group Spend Objective                                     
0-30          marketplace_campaigns            0          10317   
              portal_campaigns                 0           4193   
              am_campaigns                     0          14728   
              marketing                        0            141   
              smart_promotions                 0            605   
              bolt_plus                        0           2028   
              other_campaigns                  0             44   
              no_campaign                  11692             94   
31-60         marketplace_campaigns            0          10950   
              portal_campaigns                 0           2628   
              am_campaigns                     0           5758   
              marketing                        0            362   
              smart_promotions                 0            34

In [9]:
import pandas as pd
import numpy as np
from datetime import datetime

# ---- 0) Isolate single-order users ----
order_counts  = df_deduplicated['User ID'].value_counts()
single_users  = order_counts[order_counts == 1].index
df_single     = df_deduplicated[df_deduplicated['User ID'].isin(single_users)].copy()

# ---- 1) Last-order fields (it's their only order anyway) ----
df_single['days_since_last_order'] = (
    pd.Timestamp(datetime.today().date()) - df_single['Order Created Date']
).dt.days
df_single['used_discount'] = df_single['Discount Value Local'] > 0

# ---- 2) Recency buckets ----
bins   = [0, 30, 60, 90, 120, np.inf]
labels = ['0-30','31-60','61-90','91-120','120+']
df_single['recency_group'] = pd.cut(df_single['days_since_last_order'],
                                    bins=bins, labels=labels, right=True)

# ---- 3) Clean Spend Objective cats ----
spend_order = [
    'marketplace_campaigns','portal_campaigns','am_campaigns','marketing',
    'smart_promotions','bolt_plus','other_campaigns','no_campaign'
]
df_single['Spend Objective'] = (
    df_single['Spend Objective']
    .fillna('no_campaign')
    .astype(pd.CategoricalDtype(spend_order, ordered=True))
)

# ---- 4) Counts & % (NO filtering!) ----
counts_so_single = (
    df_single
      .groupby(['recency_group','Spend Objective','used_discount'], observed=True)
      .size()
      .unstack('used_discount', fill_value=0)
      .rename(columns={False:'No Discount', True:'Used Discount'})
)

pct_so_single = (
    counts_so_single
      .div(counts_so_single.sum(axis=1), axis=0)
      .mul(100).round(1)
      .add_suffix(' %')
)

summary_so_single = counts_so_single.join(pct_so_single)

print("=== SINGLE-ORDER USERS: summary_so_single ===")
print(summary_so_single)
print("\nAs flat df:\n", summary_so_single.reset_index())


=== SINGLE-ORDER USERS: summary_so_single ===
used_discount                        No Discount  Used Discount  \
recency_group Spend Objective                                     
0-30          marketplace_campaigns            0           2549   
              portal_campaigns                 0            519   
              am_campaigns                     0           1967   
              marketing                        0             16   
              smart_promotions                 0             15   
              bolt_plus                        0             43   
              no_campaign                   1326             17   
31-60         marketplace_campaigns            0           2957   
              portal_campaigns                 0            529   
              am_campaigns                     0           1025   
              marketing                        0            308   
              smart_promotions                 0             24   
              bo

In [11]:
OrderExperience = pd.read_csv(
    '/Users/levankikadze/Desktop/Bolt_Food/Pandas/BoltFood/churningUsers/Data/AllOrdersExperienceBreakdown.csv',
    dtype={'User ID': int, 'Order ID': int, 'Provider ID': int, 'Vendor ID': int},
    thousands=','
)

User experience Analysis

In [12]:
import pandas as pd


def prepare_order_experience_df(df_orders, df_experience_raw):
    """
   
    
    Args:
        df_orders (DataFrame): Base order-level DataFrame (e.g., single-order users)
        df_experience_raw (DataFrame): Raw experience data to join

    Returns:
        DataFrame: Cleaned and merged DataFrame
    """
    # Clean column names in experience dataset
    df_experience = df_experience_raw.copy()
    df_experience.columns = df_experience.columns.str.replace('2. Order Information ', '', regex=False)
    
    # Merge on 'Order ID'
    df_merged = pd.merge(df_orders, df_experience, on='Order ID', how='inner')

    # Handle duplicate vendor name columns
    if 'Vendor Name_y' in df_merged.columns:
        df_merged.drop(columns='Vendor Name_y', inplace=True)
    if 'Vendor Name_x' in df_merged.columns:
        df_merged.rename(columns={'Vendor Name_x': 'Vendor Name'}, inplace=True)

    # Convert date columns
    for col in df_merged.columns:
        if 'date' in col.lower():
            df_merged[col] = pd.to_datetime(df_merged[col], errors='coerce')

    # Clean numeric fields
    if 'Checkout Eta Error Seconds' in df_merged.columns:
        df_merged['Checkout Eta Error Seconds'] = pd.to_numeric(
            df_merged['Checkout Eta Error Seconds'].astype(str).str.replace(',', ''),
            errors='coerce'
        )

    if 'Eater fees per order, €' in df_merged.columns:
        df_merged['Eater fees per order, €'] = pd.to_numeric(
            df_merged['Eater fees per order, €'].astype(str).str.replace('€', ''),
            errors='coerce'
        )

    # Convert Yes/No fields to boolean
    for col in df_merged.columns:
        if 'Yes / No' in col:
            df_merged[col] = df_merged[col].str.lower().map({'yes': True, 'no': False})

    return df_merged

df_all_experience = prepare_order_experience_df(df_deduplicated, OrderExperience)
df_single_experience = prepare_order_experience_df(df_single, OrderExperience)
df_single_experience.info()

today = pd.Timestamp(datetime.today().date())

df_single_experience_churned = df_single_experience[
    (today - df_single_experience['Order Created Date']).dt.days > 30
].copy()

df_single_experience_SinceMarch = df_single_experience[
    (df_single_experience['Order Created Date'] >= pd.Timestamp('2025-03-01')) 
]



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69058 entries, 0 to 69057
Data columns (total 24 columns):
 #   Column                                           Non-Null Count  Dtype         
---  ------                                           --------------  -----         
 0   Order Created Date                               69058 non-null  datetime64[ns]
 1   Order ID                                         69058 non-null  int64         
 2   User ID                                          69058 non-null  int64         
 3   Provider ID                                      69058 non-null  int64         
 4   Provider Name                                    69058 non-null  object        
 5   Vendor ID                                        69058 non-null  int64         
 6   Vendor Name                                      69058 non-null  object        
 7   Spend Objective                                  69058 non-null  category      
 8   Discount Value Local                

In [13]:
# Step 1: Get users with 3+ orders
order_counts = df_deduplicated['User ID'].value_counts()
repeat_users_ThreeOrMore = order_counts[order_counts >= 3].index

# Step 2: Filter repeat users
df_repeat = df_deduplicated[df_deduplicated['User ID'].isin(repeat_users_ThreeOrMore)].copy()

# Step 3: Identify churned users (no order in last 30 days)
today = pd.Timestamp(datetime.today().date())
df_repeat_churned = df_repeat[
    (today - df_repeat['Order Created Date']).dt.days > 30
].copy()

# Step 4: Prepare order experience for churned users
df_repeat_experience_churned = prepare_order_experience_df(df_repeat_churned, OrderExperience)

df_repeat_Experience_sinceMarch = prepare_order_experience_df(df_repeat, OrderExperience)


# Step 5: Filter churned user experience since March
df_repeat_experience_SinceMarch = df_repeat_Experience_sinceMarch[
    df_repeat_Experience_sinceMarch['Order Created Date'] >= pd.Timestamp('2025-03-01')
].copy()

# Step 6: Filter all experience since March
df_all_experience_SinceMarch = df_all_experience[
    df_all_experience['Order Created Date'] >= pd.Timestamp('2025-03-01')
].copy()


In [14]:
# df_single_experience_SinceMarch[df_single_experience_SinceMarch['Order Created Date'] >= pd.Timestamp('2025-07-01')]
# df_repeat_experience_SinceMarch[df_repeat_experience_SinceMarch['Order Created Date'] >= pd.Timestamp('2025-03-01')]
df_all_experience_SinceMarch[
    df_all_experience_SinceMarch['Order Created Date'] < pd.Timestamp('2025-03-01')
]

,Order Created Date,Order ID,User ID,Provider ID,Provider Name,Vendor ID,Vendor Name,Spend Objective,Discount Value Local,Priority,...,Is Bad Order (Yes / No),Bad Order Type,Is Bad Courier Rating Order (Yes / No),Is Bad Provider Rating Order (Yes / No),Has Delivery Quality Cs Ticket (Yes / No),Has Timing Quality Cs Ticket (Yes / No),Has Order Quality Cs Ticket (Yes / No),Has Missing or Wrong Items Cs Ticket (Yes / No),Has Eater Cancellation Cs Ticket (Yes / No),"Eater fees per order, €"


In [16]:
def analyze_order_experience(df):
    # --- ETA Error ----------------------------------------------------------
    avg_eta_error = df['Checkout Eta Error Seconds'].mean()
    late_pct = (
        (df['Checkout Eta Error Seconds'] < -600).sum()
        / df['Checkout Eta Error Seconds'].notna().sum()
        * 100
    )
    print(f'🚚 Avg ETA error (sec): {avg_eta_error:.2f}')
    print(f'⏱️  % orders >10 min late: {late_pct:.2f}%')

    # --- Eater Fees ---------------------------------------------------------
    print(f'💶 Avg eater fee (€): {df["Eater fees per order, €"].mean():.2f}')

    # --- Ticket Analysis ----------------------------------------------------
    ticket_cols = [
        'Has Delivery Quality Cs Ticket (Yes / No)',
        'Has Timing Quality Cs Ticket (Yes / No)',
        'Has Order Quality Cs Ticket (Yes / No)',
        'Has Missing or Wrong Items Cs Ticket (Yes / No)',
        'Has Eater Cancellation Cs Ticket (Yes / No)'
    ]
    df['any_ticket_opened'] = df[ticket_cols].any(axis=1)
    total_orders = len(df)

    print(f'\n🔍 % orders with ANY CS ticket: {df["any_ticket_opened"].mean()*100:.2f}%')
    print('📊 Ticket‑type share (all orders):')
    for col in ticket_cols:
        print(f'  • {col.replace(" (Yes / No)", "")}: {df[col].mean()*100:.2f}%')

    # --- Rating Flags -------------------------------------------------------
    courier_flag  = 'Is Bad Courier Rating Order (Yes / No)'
    provider_flag = 'Is Bad Provider Rating Order (Yes / No)'

    pct_bad_courier  = df[courier_flag ].mean() * 100
    pct_bad_provider = df[provider_flag].mean() * 100
    pct_any_bad      = df[[courier_flag, provider_flag]].any(axis=1).mean() * 100

    print(f'\n⭐ % orders with BAD rating (any): {pct_any_bad:.2f}%')
    print(f'   • Courier rating bad:  {pct_bad_courier :.2f}%')
    print(f'   • Provider rating bad: {pct_bad_provider:.2f}%')

    # --- Bad Orders ---------------------------------------------------------
    bad_flag = 'Is Bad Order (Yes / No)'
    pct_bad_orders = df[bad_flag].mean() * 100
    print(f'\n🚨 % BAD orders overall: {pct_bad_orders:.2f}%')

    if df[bad_flag].any():
        bad_types = (
            df.loc[df[bad_flag], 'Bad Order Type']
              .fillna('Unknown')
              .value_counts(normalize=True)
              .mul(100)
              .round(2)
        )
        print('\n🧩 Bad‑order type split (% of BAD orders):')
        for name, pct in bad_types.items():
            print(f'  • {name}: {pct:.2f}%')

    print('\n✅ Analysis complete.\n')

        
analyze_order_experience(df_single_experience_churned)
analyze_order_experience(df_repeat_experience_churned)
analyze_order_experience(df_all_experience)


🚚 Avg ETA error (sec): -198.70
⏱️  % orders >10 min late: 20.91%
💶 Avg eater fee (€): 1.66

🔍 % orders with ANY CS ticket: 5.87%
📊 Ticket‑type share (all orders):
  • Has Delivery Quality Cs Ticket: 0.89%
  • Has Timing Quality Cs Ticket: 2.78%
  • Has Order Quality Cs Ticket: 0.64%
  • Has Missing or Wrong Items Cs Ticket: 1.14%
  • Has Eater Cancellation Cs Ticket: 0.71%

⭐ % orders with BAD rating (any): 4.74%
   • Courier rating bad:  1.31%
   • Provider rating bad: 4.08%

🚨 % BAD orders overall: 13.31%

🧩 Bad‑order type split (% of BAD orders):
  • late_delivery_order_15min: 61.17%
  • timing_quality_cs_ticket: 20.39%
  • missing_or_wrong_item_cs_ticket: 7.97%
  • delivery_quality_cs_ticket: 6.65%
  • order_quality_cs_ticket: 3.82%

✅ Analysis complete.

🚚 Avg ETA error (sec): -115.12
⏱️  % orders >10 min late: 17.04%
💶 Avg eater fee (€): 1.52

🔍 % orders with ANY CS ticket: 5.34%
📊 Ticket‑type share (all orders):
  • Has Delivery Quality Cs Ticket: 0.90%
  • Has Timing Quality Cs

In [24]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

pio.renderers.default = "browser"

# ---------- LOAD DATA ----------
orders_df = pd.read_csv(
    "/Users/levankikadze/Desktop/Bolt_Food/Pandas/BoltFood/Bad_Order_Analysis/Data/OrdersCountByVendor_Tb.csv"
)
bad_orders_df = pd.read_csv(
    "/Users/levankikadze/Desktop/Bolt_Food/Pandas/BoltFood/Bad_Order_Analysis/Data/Bad_Order_Vendors_Tb.csv"
)

# ---------- CLEAN ORDERS ----------
orders_df.columns = orders_df.columns.str.strip()
orders_df = orders_df.rename(
    columns={
        "2. Order Information Vendor Name": "VendorName",
        "2. Order Information Placed orders count, #": "OrderCount",
    }
)
orders_df["OrderCount"] = (
    orders_df["OrderCount"]
    .astype(str)
    .str.replace(r"[^\d]", "", regex=True)
    .replace("", "0")
    .astype(int)
)
orders_df = orders_df.sort_values(by="OrderCount", ascending=False)

# ---------- CLEAN BAD ORDERS ----------
bad_orders_df.columns = bad_orders_df.columns.str.strip()
bad_orders_df = bad_orders_df.rename(
    columns={
        "2. Order Information Vendor Name": "VendorName",
        "2. Order Information Is Bad Order (Yes / No)": "IsBad",
        "2. Order Information Bad Order Type": "BadOrderType",
    }
)

# ---------- METRICS ----------
top_20_vendors = orders_df.head(20)["VendorName"].tolist()
total_orders = orders_df["OrderCount"].sum()
total_bad_orders = len(bad_orders_df)

comparison_data = []
for vendor in top_20_vendors:
    vendor_orders = orders_df.loc[orders_df["VendorName"] == vendor, "OrderCount"].sum()
    order_pct = vendor_orders / total_orders * 100

    vendor_bad = len(bad_orders_df[bad_orders_df["VendorName"] == vendor])
    bad_pct = vendor_bad / total_bad_orders * 100 if total_bad_orders else 0

    comparison_data.append(
        {
            "Vendor": vendor,
            "Order Count": vendor_orders,
            "% of Total Orders": order_pct,
            "Bad Order Count": vendor_bad,
            "% of Bad Orders": bad_pct,
            "Difference (% Bad - % Total)": bad_pct - order_pct,
        }
    )

comparison_df = (
    pd.DataFrame(comparison_data)
    .sort_values(by="Difference (% Bad - % Total)", ascending=False)
)

# ---------- VISUALISATION (fig5) ----------
fig5 = make_subplots(specs=[[{"secondary_y": True}]])

# bars – total-order share
fig5.add_trace(
    go.Bar(
        x=comparison_df["Vendor"],
        y=comparison_df["% of Total Orders"],
        name="% of Total Orders",
        marker_color="#636EFA",
        opacity=0.7,
    ),
    secondary_y=False,
)

# bars – bad-order share
fig5.add_trace(
    go.Bar(
        x=comparison_df["Vendor"],
        y=comparison_df["% of Bad Orders"],
        name="% of Bad Orders",
        marker_color="#EF553B",
        opacity=0.7,
    ),
    secondary_y=False,
)

# line – difference
fig5.add_trace(
    go.Scatter(
        x=comparison_df["Vendor"],
        y=comparison_df["Difference (% Bad - % Total)"],
        name="Difference (Bad% – Total%)",
        mode="lines+markers",
        line=dict(color="#00CC96", width=2),
        marker=dict(size=8),
    ),
    secondary_y=True,
)

fig5.update_layout(
    title="<b>Fig 5 – Tbilisi Top-20 Vendors: Order Share vs Bad-Order Share</b>",
    title_x=0.5,
    xaxis_title="Vendor",
    yaxis_title="Percentage (%)",
    hovermode="x unified",
    barmode="group",
    height=600,
    template="plotly_white",
    xaxis={"categoryorder": "total descending"},
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
)

fig5.update_yaxes(title_text="<b>Percentage</b> (%)", secondary_y=False)
fig5.update_yaxes(title_text="<b>Difference</b> (percentage points)", secondary_y=True)

# ---------- SHOW CHART ----------
fig5.show()

# ---------- TEXTUAL OUTPUT ----------
print("\n🔍 Tbilisi - Top 20 Vendors: Order Share vs Bad Order Share")
print(f"📊 Total Orders: {total_orders:,}")
print(f"⚠️  Total Bad Orders: {total_bad_orders:,}\n")

print("Vendor Analysis:")
for _, row in comparison_df.iterrows():
    print(f"\n🏪 {row['Vendor']}")
    print(
        f"   📦 Orders: {row['Order Count']:,} "
        f"({row['% of Total Orders']:.2f}% of total)"
    )
    print(
        f"   ❌ Bad Orders: {row['Bad Order Count']} "
        f"({row['% of Bad Orders']:.2f}% of bad orders)"
    )
    print(
        f"   🔄 Difference: "
        f"{row['Difference (% Bad - % Total)']:+.2f}% points"
    )

    vendor_bad_orders = bad_orders_df[bad_orders_df["VendorName"] == row["Vendor"]]
    if not vendor_bad_orders.empty:
        top_reasons = vendor_bad_orders["BadOrderType"].value_counts().head(2)
        print("   Top Bad Order Reasons:")
        for reason, count in top_reasons.items():
            print(f"     - {reason}: {count}")

print("\n📈 Summary Statistics:")
print(
    f"Average % of Total Orders: "
    f"{comparison_df['% of Total Orders'].mean():.2f}%"
)
print(
    f"Average % of Bad Orders: "
    f"{comparison_df['% of Bad Orders'].mean():.2f}%"
)
print(
    f"Average Difference: "
    f"{comparison_df['Difference (% Bad - % Total)'].mean():.2f}% points"
)



🔍 Tbilisi - Top 20 Vendors: Order Share vs Bad Order Share
📊 Total Orders: 304,563
⚠️  Total Bad Orders: 30,657

Vendor Analysis:

🏪 KFC GEORGIA Tbilisi
   📦 Orders: 8,440 (2.77% of total)
   ❌ Bad Orders: 1211 (3.95% of bad orders)
   🔄 Difference: +1.18% points
   Top Bad Order Reasons:
     - late_delivery_order_15min: 370
     - failed_order_provider_rejected: 263

🏪 Fire Wok
   📦 Orders: 6,572 (2.16% of total)
   ❌ Bad Orders: 978 (3.19% of bad orders)
   🔄 Difference: +1.03% points
   Top Bad Order Reasons:
     - late_delivery_order_15min: 406
     - timing_quality_cs_ticket: 182

🏪 MC
   📦 Orders: 31,472 (10.33% of total)
   ❌ Bad Orders: 3429 (11.19% of bad orders)
   🔄 Difference: +0.85% points
   Top Bad Order Reasons:
     - missing_or_wrong_item_cs_ticket: 1126
     - late_delivery_order_15min: 953

🏪 Subway Georgia 1
   📦 Orders: 3,901 (1.28% of total)
   ❌ Bad Orders: 445 (1.45% of bad orders)
   🔄 Difference: +0.17% points
   Top Bad Order Reasons:
     - late_delivery

In [29]:
"""
Full one-page dashboard builder.

• Fig 1 – All Users: Discount vs No Discount
• Fig 2 – Single-Order Users: Discount vs No Discount
• Fig 3 – All Users: Spend Objective split
• Fig 4 – Single-Order Users: Spend Objective split
• Fig 5 – Top-20 Vendors: Order-share vs Bad-order share
• 2 KPI tables + narrative

Data requirements:
  - counts, pct, counts_single, pct_single, counts_so, counts_so_single
    (these DataFrames come from your earlier analysis code)
  - OrdersCountByVendor_Tb.csv
  - Bad_Order_Vendors_Tb.csv
"""

# ---------- IMPORTS ----------
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import re
from pathlib import Path

pio.renderers.default = "browser"

# =======================================================================
# SECTION A – FIG 5 DATA (Vendor bad-order analysis)
# =======================================================================

ORDERS_PATH = Path(
    "/Users/levankikadze/Desktop/Bolt_Food/Pandas/BoltFood/Bad_Order_Analysis/Data/OrdersCountByVendor_Tb.csv"
)
BAD_ORDERS_PATH = Path(
    "/Users/levankikadze/Desktop/Bolt_Food/Pandas/BoltFood/Bad_Order_Analysis/Data/Bad_Order_Vendors_Tb.csv"
)

orders_df = pd.read_csv(ORDERS_PATH)
bad_orders_df = pd.read_csv(BAD_ORDERS_PATH)

# Clean orders
orders_df.columns = orders_df.columns.str.strip()
orders_df = orders_df.rename(
    columns={
        "2. Order Information Vendor Name": "VendorName",
        "2. Order Information Placed orders count, #": "OrderCount",
    }
)
orders_df["OrderCount"] = (
    orders_df["OrderCount"]
    .astype(str)
    .str.replace(r"[^\d]", "", regex=True)
    .replace("", "0")
    .astype(int)
)
orders_df = orders_df.sort_values(by="OrderCount", ascending=False)

# Clean bad orders
bad_orders_df.columns = bad_orders_df.columns.str.strip()
bad_orders_df = bad_orders_df.rename(
    columns={
        "2. Order Information Vendor Name": "VendorName",
        "2. Order Information Is Bad Order (Yes / No)": "IsBad",
        "2. Order Information Bad Order Type": "BadOrderType",
    }
)

top_20_vendors = orders_df.head(20)["VendorName"].tolist()
total_orders = orders_df["OrderCount"].sum()
total_bad_orders = len(bad_orders_df)

comparison_rows = []
for vendor in top_20_vendors:
    vendor_orders = orders_df.loc[orders_df["VendorName"] == vendor, "OrderCount"].sum()
    order_pct = vendor_orders / total_orders * 100

    vendor_bad = len(bad_orders_df[bad_orders_df["VendorName"] == vendor])
    bad_pct = vendor_bad / total_bad_orders * 100 if total_bad_orders else 0

    comparison_rows.append(
        {
            "Vendor": vendor,
            "Order Count": vendor_orders,
            "% of Total Orders": order_pct,
            "Bad Order Count": vendor_bad,
            "% of Bad Orders": bad_pct,
            "Diff (Bad%-Total%)": bad_pct - order_pct,
        }
    )

comparison_df = (
    pd.DataFrame(comparison_rows)
    .sort_values(by="Diff (Bad%-Total%)", ascending=False)
)

# ---------- FIG 5 ----------
fig5 = make_subplots(specs=[[{"secondary_y": True}]])

fig5.add_trace(
    go.Bar(
        x=comparison_df["Vendor"],
        y=comparison_df["% of Total Orders"],
        name="% of Total Orders",
        marker_color="#636EFA",
        opacity=0.7,
    ),
    secondary_y=False,
)
fig5.add_trace(
    go.Bar(
        x=comparison_df["Vendor"],
        y=comparison_df["% of Bad Orders"],
        name="% of Bad Orders",
        marker_color="#EF553B",
        opacity=0.7,
    ),
    secondary_y=False,
)
fig5.add_trace(
    go.Scatter(
        x=comparison_df["Vendor"],
        y=comparison_df["Diff (Bad%-Total%)"],
        name="Difference (Bad% – Total%)",
        mode="lines+markers",
        line=dict(color="#00CC96", width=2),
        marker=dict(size=8),
    ),
    secondary_y=True,
)
fig5.update_layout(
    title="<b>Fig 5 – Top-20 Vendors: Order Share vs Bad-Order Share</b>",
    title_x=0.5,
    xaxis_title="Vendor",
    yaxis_title="Percentage (%)",
    hovermode="x unified",
    barmode="group",
    height=600,
    template="plotly_white",
    xaxis={"categoryorder": "total descending"},
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
)
fig5.update_yaxes(title_text="<b>Percentage</b> (%)", secondary_y=False)
fig5.update_yaxes(title_text="<b>Difference</b> (pp)", secondary_y=True)

html5 = pio.to_html(fig5, include_plotlyjs=False, full_html=False)

vendor_lines = [
    f"<p><b>Total Orders:</b> {total_orders:,}</p>",
    f"<p><b>Total Bad Orders:</b> {total_bad_orders:,}</p>",
    "<h4>Vendor Breakdown</h4>",
    "<ul>",
]
for _, r in comparison_df.iterrows():
    vendor_lines.append(
        f"<li><b>{r['Vendor']}</b>: "
        f"{r['Order Count']:,} orders "
        f"({r['% of Total Orders']:.2f}%) – "
        f"{r['Bad Order Count']} bad "
        f"({r['% of Bad Orders']:.2f}%, "
        f"Δ {r['Diff (Bad%-Total%)']:+.2f} pp)</li>"
    )
vendor_lines.append("</ul>")
vendor_analysis_html = "\n".join(vendor_lines)

# =======================================================================
# SECTION B – FIG 1-4 (you already compute counts / pct DataFrames)
# =======================================================================

RECENCY_ORDER_BOTTOM_FIRST = ["120+", "91-120", "61-90", "31-60", "0-30"]
RECENCY_ORDER_TOP_FIRST = RECENCY_ORDER_BOTTOM_FIRST[::-1]
LABEL_HIDE_THRESHOLD = 3

def make_discount_vs_nodiscount_fig(counts_df, pct_df, title, x_label):
    plot_df = counts_df.copy()
    for col in ["No Discount", "Used Discount"]:
        plot_df[col + " %"] = pct_df[col + " %"]

    long = (
        plot_df.reset_index()
        .melt(
            id_vars="recency_group",
            value_vars=["No Discount", "Used Discount"],
            var_name="discount_flag",
            value_name="count",
        )
    )
    pct_long = (
        plot_df.reset_index()
        .melt(
            id_vars="recency_group",
            value_vars=["No Discount %", "Used Discount %"],
            var_name="discount_flag_pct",
            value_name="pct",
        )
    )
    long["pct"] = pct_long["pct"].values
    long["recency_group"] = pd.Categorical(
        long["recency_group"], RECENCY_ORDER_BOTTOM_FIRST, ordered=True
    )

    fig = go.Figure()
    for flag, chunk in long.groupby("discount_flag", observed=True):
        texts = chunk["pct"].astype(str) + "%"
        texts = [
            t if float(t.strip("%")) >= LABEL_HIDE_THRESHOLD else "" for t in texts
        ]
        fig.add_trace(
            go.Bar(
                y=chunk["recency_group"],
                x=chunk["count"],
                name=flag,
                orientation="h",
                text=texts,
                textposition="inside",
                insidetextanchor="middle",
                hovertemplate=(
                    "Recency: %{y}<br>"
                    + flag
                    + " count: %{x}<br>"
                    + "Share: %{text}<extra></extra>"
                ),
            )
        )

    fig.update_layout(
        barmode="stack",
        title=f"<b>{title}</b>",
        xaxis_title=x_label,
        yaxis_title="Recency Group (days since last order)",
        hovermode="y",
        legend=dict(
            orientation="h",
            yanchor="top",
            y=-0.25,
            xanchor="center",
            x=0.5,
        ),
        margin=dict(l=90, r=40, t=70, b=150),
    )
    fig.update_yaxes(categoryorder="array", categoryarray=RECENCY_ORDER_TOP_FIRST)
    return fig

def collapse_so(counts_so_df):
    collapsed = counts_so_df.sum(axis=1).rename("count").reset_index()
    collapsed["pct_bar"] = (
        collapsed["count"]
        / collapsed.groupby("recency_group", observed=True)["count"].transform("sum")
    ).mul(100).round(1)
    collapsed["label"] = collapsed["pct_bar"].astype(str) + "%"
    collapsed.loc[collapsed["pct_bar"] < LABEL_HIDE_THRESHOLD, "label"] = ""
    collapsed["recency_group"] = pd.Categorical(
        collapsed["recency_group"], RECENCY_ORDER_BOTTOM_FIRST, ordered=True
    )
    return collapsed

def make_spend_objective_fig(collapsed_df, title, x_label):
    fig = go.Figure()
    for so, df_so in collapsed_df.groupby("Spend Objective", observed=True):
        fig.add_trace(
            go.Bar(
                y=df_so["recency_group"],
                x=df_so["count"],
                name=so,
                orientation="h",
                text=df_so["label"],
                textposition="inside",
                insidetextanchor="middle",
                customdata=np.stack([df_so["pct_bar"]], axis=-1),
                hovertemplate=(
                    "Recency: %{y}<br>"
                    "Spend Objective: "
                    + so
                    + "<br>"
                    "Count: %{x}<br>"
                    "Share of bar: %{customdata[0]}%<extra></extra>"
                ),
            )
        )

    fig.update_layout(
        barmode="stack",
        title=f"<b>{title}</b>",
        xaxis_title=x_label,
        yaxis_title="Recency Group (days since last order)",
        hovermode="y",
        legend=dict(
            orientation="h",
            yanchor="top",
            y=-0.28,
            xanchor="center",
            x=0.5,
            traceorder="reversed",
        ),
        margin=dict(l=110, r=40, t=70, b=150),
    )
    fig.update_yaxes(categoryorder="array", categoryarray=RECENCY_ORDER_TOP_FIRST)

    buttons = []
    all_vis = [True] * len(fig.data)
    buttons.append(
        dict(
            label="All",
            method="update",
            args=[{"visible": all_vis}, {"title": f"<b>{title} — All</b>"}],
        )
    )
    for so in collapsed_df["Spend Objective"].unique():
        vis = [tr.name == so for tr in fig.data]
        buttons.append(
            dict(
                label=so,
                method="update",
                args=[
                    {"visible": vis},
                    {"title": f"<b>{title} — {so}</b>"},
                ],
            )
        )
    fig.update_layout(
        updatemenus=[
            dict(
                type="dropdown",
                x=1.02,
                y=1,
                xanchor="left",
                yanchor="top",
                buttons=buttons,
                showactive=True,
            )
        ]
    )
    return fig

# Build Figs 1-4
fig1 = make_discount_vs_nodiscount_fig(
    counts, pct, "Discount vs No Discount by Recency Group (All Users)", "Count"
)
fig2 = make_discount_vs_nodiscount_fig(
    counts_single, pct_single, "Discount vs No Discount by Recency Group (Single-Order Users)", "Count"
)
fig3 = make_spend_objective_fig(
    collapse_so(counts_so), "Last-Order Split by Spend Objective (All Users)", "Count"
)
fig4 = make_spend_objective_fig(
    collapse_so(counts_so_single), "Last-Order Split by Spend Objective (Single-Order Users)", "Count"
)

html1 = pio.to_html(fig1, include_plotlyjs="cdn", full_html=False)
html2 = pio.to_html(fig2, include_plotlyjs=False, full_html=False)
html3 = pio.to_html(fig3, include_plotlyjs=False, full_html=False)
html4 = pio.to_html(fig4, include_plotlyjs=False, full_html=False)

# =======================================================================
# SECTION C – KPI TABLES & NARRATIVE (unchanged from your original)
# =======================================================================

description = """
<div style="max-width: 900px; margin: 0;">
  <h3>What the Analysis Covers:</h3>
  <ul>
    <li><b>1. Order Timeliness:</b>
      <ul>
        <li>Calculates the average delivery delay across all orders.</li>
        <li>Flags the % of orders that were more than 10 minutes late.</li>
      </ul>
    </li>
    <li><b>2. Customer Fees:</b>
      <ul>
        <li>Computes the average fee paid by customers per order.</li>
      </ul>
    </li>
    <li><b>3. Customer Support Issues:</b>
      <ul>
        <li>Measures how often support tickets are raised.</li>
        <li>Breaks down the ticket reasons: delivery, timing, wrong items, etc.</li>
      </ul>
    </li>
    <li><b>4. Customer Ratings:</b>
      <ul>
        <li>Tracks % of orders with bad ratings.</li>
        <li>Separates courier vs. provider-related rating issues.</li>
      </ul>
    </li>
    <li><b>5. Bad Orders:</b>
      <ul>
        <li>Shows % of all orders flagged as “bad.”</li>
        <li>Breaks down key reasons like lateness and CS issues.</li>
      </ul>
    </li>
  </ul>
</div>
"""

table1_html = """
<table style="border-collapse:collapse; margin:20px auto;">
  <thead>
    <tr>
      <th style="border:1px solid #ddd; padding:6px;">Cohort</th>
      <th style="border:1px solid #ddd; padding:6px;">Avg ETA<br>error (s)</th>
      <th style="border:1px solid #ddd; padding:6px;">% >10 min<br>late</th>
      <th style="border:1px solid #ddd; padding:6px;">Avg eater<br>fee (€)</th>
      <th style="border:1px solid #ddd; padding:6px;">% with<br>any CS ticket</th>
      <th style="border:1px solid #ddd; padding:6px;">Delivery<br>Quality (%)</th>
      <th style="border:1px solid #ddd; padding:6px;">Timing<br>Quality (%)</th>
      <th style="border:1px solid #ddd; padding:6px;">Order<br>Quality (%)</th>
      <th style="border:1px solid #ddd; padding:6px;">Missing/Wrong<br>Items (%)</th>
      <th style="border:1px solid #ddd; padding:6px;">Eater<br>Cancel (%)</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="border:1px solid #ddd; padding:6px;">1× Churned<br>(>30 d, 1 order)</td>
      <td style="border:1px solid #ddd; padding:6px;">–200.08</td>
      <td style="border:1px solid #ddd; padding:6px;">33.90%</td>
      <td style="border:1px solid #ddd; padding:6px;">1.66</td>
      <td style="border:1px solid #ddd; padding:6px;">5.88%</td>
      <td style="border:1px solid #ddd; padding:6px;">0.89%</td>
      <td style="border:1px solid #ddd; padding:6px;">2.79%</td>
      <td style="border:1px solid #ddd; padding:6px;">0.65%</td>
      <td style="border:1px solid #ddd; padding:6px;">1.14%</td>
      <td style="border:1px solid #ddd; padding:6px;">0.72%</td>
    </tr>
    <tr>
      <td style="border:1px solid #ddd; padding:6px;">3+ Churned<br>(>30 d, ≥3 orders)</td>
      <td style="border:1px solid #ddd; padding:6px;">–115.60</td>
      <td style="border:1px solid #ddd; padding:6px;">30.20%</td>
      <td style="border:1px solid #ddd; padding:6px;">1.52</td>
      <td style="border:1px solid #ddd; padding:6px;">5.34%</td>
      <td style="border:1px solid #ddd; padding:6px;">0.90%</td>
      <td style="border:1px solid #ddd; padding:6px;">2.25%</td>
      <td style="border:1px solid #ddd; padding:6px;">0.62%</td>
      <td style="border:1px solid #ddd; padding:6px;">1.58%</td>
      <td style="border:1px solid #ddd; padding:6px;">0.29%</td>
    </tr>
    <tr>
      <td style="border:1px solid #ddd; padding:6px;">All Users</td>
      <td style="border:1px solid #ddd; padding:6px;">–118.85</td>
      <td style="border:1px solid #ddd; padding:6px;">30.23%</td>
      <td style="border:1px solid #ddd; padding:6px;">1.53</td>
      <td style="border:1px solid #ddd; padding:6px;">5.35%</td>
      <td style="border:1px solid #ddd; padding:6px;">0.91%</td>
      <td style="border:1px solid #ddd; padding:6px;">2.22%</td>
      <td style="border:1px solid #ddd; padding:6px;">0.64%</td>
      <td style="border:1px solid #ddd; padding:6px;">1.56%</td>
      <td style="border:1px solid #ddd; padding:6px;">0.33%</td>
    </tr>
  </tbody>
</table>
"""

table2_html = """
<table style="border-collapse:collapse; margin:20px auto;">
  <thead>
    <tr>
      <th style="border:1px solid #ddd; padding:6px;">Cohort</th>
      <th style="border:1px solid #ddd; padding:6px;">Any<br>Bad Rating</th>
      <th style="border:1px solid #ddd; padding:6px;">Courier<br>Bad (%)</th>
      <th style="border:1px solid #ddd; padding:6px;">Provider<br>Bad (%)</th>
      <th style="border:1px solid #ddd; padding:6px;">% Bad<br>Orders</th>
      <th style="border:1px solid #ddd; padding:6px;">Late ≥15min<br>of Bad (%)</th>
      <th style="border:1px solid #ddd; padding:6px;">Timing CS<br>of Bad (%)</th>
      <th style="border:1px solid #ddd; padding:6px;">Missing/Wrong<br>of Bad (%)</th>
      <th style="border:1px solid #ddd; padding:6px;">Delivery CS<br>of Bad (%)</th>
      <th style="border:1px solid #ddd; padding:6px;">Order CS<br>of Bad (%)</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="border:1px solid #ddd; padding:6px;">1× Churned<br>(>30 d, 1 order)</td>
      <td style="border:1px solid #ddd; padding:6px;">4.74%</td>
      <td style="border:1px solid #ddd; padding:6px;">1.32%</td>
      <td style="border:1px solid #ddd; padding:6px;">4.08%</td>
      <td style="border:1px solid #ddd; padding:6px;">13.33%</td>
      <td style="border:1px solid #ddd; padding:6px;">61.18%</td>
      <td style="border:1px solid #ddd; padding:6px;">20.44%</td>
      <td style="border:1px solid #ddd; padding:6px;">7.95%</td>
      <td style="border:1px solid #ddd; padding:6px;">6.62%</td>
      <td style="border:1px solid #ddd; padding:6px;">3.82%</td>
    </tr>
    <tr>
      <td style="border:1px solid #ddd; padding:6px;">3+ Churned<br>(>30 d, ≥3 orders)</td>
      <td style="border:1px solid #ddd; padding:6px;">4.09%</td>
      <td style="border:1px solid #ddd; padding:6px;">0.96%</td>
      <td style="border:1px solid #ddd; padding:6px;">3.56%</td>
      <td style="border:1px solid #ddd; padding:6px;">10.83%</td>
      <td style="border:1px solid #ddd; padding:6px;">53.52%</td>
      <td style="border:1px solid #ddd; padding:6px;">20.01%</td>
      <td style="border:1px solid #ddd; padding:6px;">13.76%</td>
      <td style="border:1px solid #ddd; padding:6px;">8.20%</td>
      <td style="border:1px solid #ddd; padding:6px;">4.51%</td>
    </tr>
    <tr>
      <td style="border:1px solid #ddd; padding:6px;">All Users</td>
      <td style="border:1px solid #ddd; padding:6px;">4.15%</td>
      <td style="border:1px solid #ddd; padding:6px;">1.00%</td>
      <td style="border:1px solid #ddd; padding:6px;">3.61%</td>
      <td style="border:1px solid #ddd; padding:6px;">10.90%</td>
      <td style="border:1px solid #ddd; padding:6px;">54.17%</td>
      <td style="border:1px solid #ddd; padding:6px;">19.59%</td>
      <td style="border:1px solid #ddd; padding:6px;">13.50%</td>
      <td style="border:1px solid #ddd; padding:6px;">8.24%</td>
      <td style="border:1px solid #ddd; padding:6px;">4.49%</td>
    </tr>
  </tbody>
</table>
"""

analysis_text_html = """
<h2>Observations &amp; Take-aways</h2>
<ul>
<li><b>First-order experience is the weakest.</b> One-time churners suffer the
worst operational performance: longest ETA error (-200 s), highest late-delivery
rate (34 %), and the largest share of “BAD” orders (13 %). More than 60 % of
their bad orders are purely 15-minute late deliveries.</li>
<li><b>Provider quality drives dissatisfaction more than courier quality.</b>
Bad-provider ratings appear on 4.1 % of one-timer orders—about 15 % higher than
the other cohorts. Courier-rating issues remain under 1 % everywhere, so
provider-side factors (prep time, packaging, item accuracy) are the bigger
leverage.</li>
<li><b>Customer-service tickets mirror the delay pattern.</b> Timing-quality
tickets dominate each cohort but peak in one-time churners (2.8 % vs 2.2 %
baseline). Delivery-quality and missing/wrong-item tickets also climb for
one-timers, highlighting that the very first interaction exposes multiple pain
points.</li>
<li><b>Price perception is unlikely the root cause.</b> Average eater fees
differ by just €0.14 between the worst and best cohorts—operational pain, not
fees, is the churn trigger.</li>
</ul>
"""

# =======================================================================
# SECTION D – FINAL HTML RENDER
# =======================================================================

nav_html = """
<b>Jump to:</b><br>
<a href="#fig1">1. All Users – Disc vs No Disc</a><br>
<a href="#fig2">2. Single Order – Disc vs No Disc</a><br>
<a href="#fig3">3. All Users – Spend Objective</a><br>
<a href="#fig4">4. Single Order – Spend Objective</a><br>
<a href="#fig5">5. Vendor Bad Orders</a><br>
<a href="#analysis">6. Comparative Analysis</a>
"""

DASHBOARD_FILE = "discount_dashboard.html"

full_html = f"""
<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>Discount Dashboard</title>
<style>
body {{ font-family: Arial, sans-serif; margin: 0 20px 40px; }}
h1  {{ margin-top: 30px; }}
.nav {{
  position: fixed; top: 10px; right: 10px;
  background: #f7f7f7; padding: 10px 14px;
  border: 1px solid #ddd; border-radius: 8px;
  font-size: 14px; line-height: 1.6;
}}
.nav a {{ text-decoration: none; color: #0077cc; }}
.nav a:hover {{ text-decoration: underline; }}
.section {{ margin: 80px 0; }}
#analysis table {{
  border-collapse: collapse;
  margin: 20px auto;
}}
#analysis th, #analysis td {{
  border: 1px solid #ddd;
  padding: 6px;
  text-align: center;
}}
</style>
</head>
<body>

<div class="nav">{nav_html}</div>

<h1>Discount Dashboard – Timeframe: Jul 29 2024 – Jul 29 2025</h1>

<div id="fig1" class="section">
  <h2>1. All Users – Discount vs No Discount</h2>
  {html1}
</div>

<div id="fig2" class="section">
  <h2>2. Single-Order Users – Discount vs No Discount</h2>
  {html2}
</div>

<div id="fig3" class="section">
  <h2>3. All Users – Split by Spend Objective</h2>
  {html3}
</div>

<div id="fig4" class="section">
  <h2>4. Single-Order Users – Split by Spend Objective</h2>
  {html4}
</div>

<div id="fig5" class="section">
  <h2>5. Vendor Bad Orders – Top 20</h2>
  {html5}
  <br><br>
  {vendor_analysis_html}
</div>

<div id="analysis" class="section">
  {description}<br>
  {table1_html}<br>
  {table2_html}<br>
  {analysis_text_html}
</div>

</body>
</html>
"""

with open(DASHBOARD_FILE, "w", encoding="utf-8") as f:
    f.write(full_html)

print(f"✅  Saved dashboard to {DASHBOARD_FILE}")


✅  Saved dashboard to discount_dashboard.html
